# GMS Authentication

## Background

As part of NGIS Genomics England are migrating from LDAP and adopting Active Directory (AD). This means that the authentication in the Interpretation API (aka CIP-API ) will also be moved to AD. 

This will impact the  way systems such as internal pipelines, interpretation services (external and internal) and decision support systems (DSS) authenticate against the CIP-API. 

The following notebook provides an example of how to use Python to obtain an AD Authentication token for system access to the CIP-API aka "Client Credentials Flow" 

### Authentication Method

The authentication method will be the same we are using at the moment, based on a `Bearer Token` present in the header of each request. 

This token will have an expiration time, clients should get a new one when this happens. 


In [3]:
tenant_id = "afee026d-8f37-400e-8869-72d9124873e4" # this is the ID of the Active Dirctory "tenant"
client_id = "" # this is the ID of the client (e.g. a gel2mdt application) that is going to query the CIP-API
client_secret = "" # this is the password for the client that is going to query the CIP-API

_**If you are developing (or working with) an application (e.g. gel2mdt) that requires system access to the CIP-API please contact Genomics England Service Desk to obtain details of the `tenant_id`, `client_id` and `client_sceret`**_

In [7]:
import requests
 
url = "https://login.microsoftonline.com/{tenant_id}/oauth2/token".format(tenant_id=tenant_id)
 
payload = "grant_type=client_credentials&client_id={client_id}&client_secret={client_secret}".format(client_id=client_id,
                                                                                                     client_secret=client_secret)
headers = {
    'Content-Type': "application/x-www-form-urlencoded",
    }
 
response = requests.request("POST", url, data=payload, headers=headers).json()
 
# print(response.text)

In [9]:
token = response['access_token']

In [10]:
token

'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCIsImtpZCI6ImllX3FXQ1hoWHh0MXpJRXN1NGM3YWNRVkduNCJ9.eyJhdWQiOiIwMDAwMDAwMi0wMDAwLTAwMDAtYzAwMC0wMDAwMDAwMDAwMDAiLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC9hZmVlMDI2ZC04ZjM3LTQwMGUtODg2OS03MmQ5MTI0ODczZTQvIiwiaWF0IjoxNTY2MzE2NDA0LCJuYmYiOjE1NjYzMTY0MDQsImV4cCI6MTU2NjMyMDMwNCwiYWlvIjoiNDJGZ1lLamNkSXhwVWY4WGQvVTh1OFk3N3YzY0FBPT0iLCJhcHBpZCI6IjgxNjQ1MzZmLTc2Y2ItNGRjNS04OGE1LWUzOTZiYmI1NjVlMCIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0L2FmZWUwMjZkLThmMzctNDAwZS04ODY5LTcyZDkxMjQ4NzNlNC8iLCJvaWQiOiIyODQyNWY2ZS1iN2EyLTQ0YjEtODg4Mi00ODdlY2EwMDJmNTUiLCJzdWIiOiIyODQyNWY2ZS1iN2EyLTQ0YjEtODg4Mi00ODdlY2EwMDJmNTUiLCJ0ZW5hbnRfcmVnaW9uX3Njb3BlIjoiRVUiLCJ0aWQiOiJhZmVlMDI2ZC04ZjM3LTQwMGUtODg2OS03MmQ5MTI0ODczZTQiLCJ1dGkiOiJHNHV3TWs5SlhVT3RvNjl0aGtXd0FBIiwidmVyIjoiMS4wIn0.T21f0TMEIkMjd-AO9kVUF6cXnQqVJaNbjAMCSyaT5EILObWQwxKoI48dN2QbADv5zqMIJKlc4cabEyX-wqdZFS3oqC9cPvYt2TRMKpkJfgdfcrlIz7VzKG3VbbyufK6n-d7ETysEjd

## Example using AD authentication token with requests library

In [16]:
interpretation_request_id = "88"
interpretation_request_version = "1"

In [22]:
def get_ir(token, interpretation_request_id, interpretation_request_version, extra_params="?extra-params=show_referral"):
    
    """ Get the complete interpretation request data for a specific case and version 
    
    uses an extra parameter to get the referral information for GMS cases
    
    """
    
    url = '{host}/api/2/interpretation-request/{ir_id}/{ir_v}/{extras}'.format(host=host, 
                                                                               ir_id=interpretation_request_id, 
                                                                               ir_v=interpretation_request_version, 
                                                                               extras=extra_params)
    auth_header = {'Authorization': 'JWT {}'.format(token)}
    tv = requests.get(url, headers=auth_header)
    
    return tv

    # check that the ir has the interpretation request data
#     if 'interpretation_request_data' in tv and tv['interpretation_request_data']:
#         return tv
#     else:
#         raise Exception('Case not found')

In [23]:
ir = get_ir(token=token, interpretation_request_id=interpretation_request_id, 
            interpretation_request_version=interpretation_request_version)

In [27]:
ir

<Response [403]>

In [28]:
ir.json()

{'detail': 'This action can not be performed by Interpretation Services or Decision Support Systems'}

## Example using AD authentication token with pycipapi client

In [12]:
host = "https://cipapi-gms-beta.gel.zone"
from pycipapi.cipapi_client import CipApiClient
cipapi = CipApiClient(host, token=token) # this creates a cip-api client which can then be used to make additional queries of the CIP-API data

In [14]:
for referral in cipapi.list_referral():
    print('Referral id: {}'.format(referral.referral_id))

Referral id: r19552775693
Referral id: r19754290876
Referral id: r19448207345
Referral id: r19156653850
Referral id: r19337397961
Referral id: r19698886184
Referral id: r19392802654
Referral id: r19628951458
Referral id: r19087814411
Referral id: r19475647240
Referral id: r19239498445
Referral id: r19281993271
Referral id: r19016784391
Referral id: r19990439671
Referral id: r19559016721
Referral id: r19031314434
Referral id: r19888729392
Referral id: r19804264624
Referral id: r19066758142
Referral id: r19879630291
Referral id: r19935034984
Referral id: r19252933191
Referral id: r19614421412
Referral id: r19585361336
Referral id: r19407332690
Referral id: r19503612036
Referral id: r19498181097
Referral id: r19806980097
Referral id: r19226588580
Referral id: r19643481497
Referral id: r19206627604
Referral id: r19115779204
Referral id: r19182998460
Referral id: r19238403159
Referral id: r19143076503
Referral id: r19102201856
Referral id: r19518142077
Referral id: r19002254359
Referral id: